## 기본양식에서 내가 원하는 슬라이드 가져와서 붙여넣기
-> 원래 이렇게 하려고 했으나, 기본 양식에서 슬라이드 창만 복사해서 새로운 슬라이드에 붙는 것이 안됨.

In [ ]:
def duplicate_slide(prs):
    source_slide = prs.slides[0]
    slide_layout = prs.slide_layouts[-1]
    copied_slide = prs.slides.add_slide(slide_layout)
    
    # 원본 슬라이드에서 shape 복제
    for shape in source_slide.shapes:
        el = shape.element
        newel = copy.deepcopy(el)
        copied_slide.shapes._spTree.insert_element_before(newel, 'p:extLst')
        
    return copied_slide

duplicate_slide(prs)
prs.save(folder_path + '/tmp.pptx')      # -> prs로 저장할 경우 원본이 복사된 후 내가 원하는 슬라이드가 복사가 됨. 따라서 
new_prs.save(folder_path + '/tmp.pptx')  # -> new_prs로 저장할 경우 새로 presentation을 만들기 때문에 오류가 발생해서 복구하라는 창이 나옴

'''
for shape in source_slide.shapes은 내가 복사하고자 하는 슬라이드 창의 도형을 가지고 옴.
결과적으로 위의 코드는 내가 원했던, 기본양식.ppt에서 원하는 슬라이드 창을 복사해서 새로운 ppt에 붙이는 방식이 안됨.
따라서 prs.save로 ppt이름만 변경한 후에 해당 ppt 안에서 슬라이드 창 복사를 진행해야함.
슬라이드를 다 만들었을 경우, slides[2], [3]번 창은 삭제를 해주기
(이미지, 영상 넣는 기본 양식에 맞게 꾸며둔 것이기 때문에, 슬라이드를 만들 때는 복사를 해야해서 필요하다.
하지만 슬라이드를 다 만들고 난 뒤에는 필요가 없기 때문에 슬라이드를 삭제해주어야 한다.)
'''

In [1]:
import cv2
import os
from pptx import Presentation
from pptx.util import Inches, Cm, Pt        # 크기 조정(사진, 클씨, 영상 등 모든 것)
from pptx.enum.text import PP_ALIGN   # 글씨 정렬
from pptx.dml.color import RGBColor   # 색깔

import copy
from copy import deepcopy
import nbformat   # 다른 ipybn파일 불러오기

In [2]:
folder_path = 'C:/Users/JungHyeona/Documents/Project/pptx'
basic_prs = Presentation(folder_path + '/basic.pptx')

In [3]:
basic_prs.save(folder_path + '/tmp.pptx')

In [4]:
image_file = [[folder_path + '/image/b/banana_01.jpg', folder_path+'/image/b/banana_02.jpg']]
video_path = [folder_path + '/image/b/reeds.mp4', folder_path + '/image/b/ocean.mp4']

In [5]:
# 기본양식을 복사하고 이름만 바꿔서 저장.
# csv파일로 받을 것이기 때문에, csv파일 열고, 해당 이름을 tmp 대신에 넣어주기
prs = Presentation(folder_path + '/tmp.pptx')

### 슬라이드 복사
https://copyprogramming.com/howto/python-pptx-adding-entire-slide-from-another-presentation

### 이미지

In [6]:
def slide_num(prs, index):
    source_slide = prs.slides[index]
    slide_layout = source_slide.slide_layout
    copied_slide = prs.slides.add_slide(slide_layout)

    # 원본 슬라이드에서 shape 복제
    for shape in source_slide.shapes:
        el = shape.element
        newel = copy.deepcopy(el)
        copied_slide.shapes._spTree.insert_element_before(newel, 'p:extLst')

    return source_slide

In [7]:
# 이미지 넣는 슬라이드 복사
def copy_slide02(prs):

    # 복사해서 가져오고 싶은 번호 지정
    slide_num(prs, 2)

    #return copied_slide

In [8]:
def insert_image(prs, image_file):

    
    # 이미지 넣을 슬라이드(맨 마지막 슬라이드에 들어가도록)
    slide = prs.slides[-1]
    
    for sublist in image_file:

        # 이미지 왼쪽
        # left : x좌표 , top : y좌표 , width : 이미지 가로 길이 , height : 이미지 세로 길이
        slide.shapes.add_picture(sublist[0], left=Cm(2), top=Cm(8), width=Cm(20), height=Cm(15))

        # 이미지 오른쪽
        slide.shapes.add_picture(sublist[1], left=Cm(23), top=Cm(8), width=Cm(20), height=Cm(15))

### 영상

In [17]:
# 영상 넣는 슬라이드 복사
def copy_slide03(prs):
    
    # 복사해서 가져오고 싶은 번호 지정
    slide_num(prs, 3)
    

In [20]:
# 폴더 내의 영상 파일들을 순회
def capture_video_thumbnail(video_path, thumbnail_path):
    
    for image_file in range(len(video_path)-1):
        video = cv2.VideoCapture(video_path[image_file])
        ret, frame = video.read()

        if ret:
            cv2.imwrite(thumbnail_path, frame)
        video.release()   # 비디오 캡처 객체 해제
        
    return thumbnail_path

In [25]:
video_path[1]

'C:/Users/JungHyeona/Documents/Project/pptx/image/b/ocean.mp4'

In [21]:
# PPT에 동영상 슬라이드를 추가하고, 동영상 썸네일을 삽입하는 함수
def insert_video_slide(prs, video_path, thumbnail_path):
    # 이미지 넣을 슬라이드(맨 마지막 슬라이드에 들어가도록)
    slide = prs.slides[-1]
    
    # 동영상에 썸네일 넣고, 동영상 삽입
    # 왼쪽에 들어가는 영상
    slide.shapes.add_movie(video_path[0], left=Cm(1), top=Cm(3.8), width=Cm(20), height=Cm(15)
                                    , poster_frame_image=thumbnail_path)

    # 오른쪽에 들어가는 영상
    slide.shapes.add_movie(video_path[1], left=Cm(26.3), top=Cm(3.9), width=Cm(18.27), height=Cm(13.7)
                                    , poster_frame_image=thumbnail_path)
    
    # 영상 자동 재생
    slide.shapes._spTree.remove(slide.shapes._spTree[-1])
    
    # 영상 썸네일 이미지 삭제
    os.remove(thumbnail_path)
    return slide

In [22]:
# 폴더 내의 영상 파일들을 PPT에 슬라이드로 추가하는 함수
def create_video(prs, video_path):

    for image_file in video_path:
        video_name = image_file.split("/")[-1]
        thumbnail_path = os.path.join(folder_path, f'{os.path.splitext(image_file)[0]}.png')
        
        # 영상 썸네일 이미지를 캡쳐하여 저장
        capture_video_thumbnail(video_path, thumbnail_path)

        # 영상 삽입
        insert_video_slide(prs, video_path, thumbnail_path)

### 값이 이미지인지, 영상인지에 따라서 해당하는 함수로 넘어가서 돌아갈 수 있게 하기

In [ ]:
for img_list in image_file:
    print(img_list[-4])


### 사용자 입력

In [9]:
def str_in(prs, name, date):

    #for i in range(len(prs.slides)):
    slide = prs.slides[-1]
    
    # 모든 페이지에 공통으로 들어가는 조건
    # update 시간
    update_layout = slide.shapes.add_textbox(left=Cm(40.84), top=Cm(2.22), width=Cm(7.25), height=Cm(0.94))
    update = update_layout.text_frame
    update.text = '(Fin. update : ' + date + ')'
    # title.text_frame.parhagraphs[0].font.name =    # font 이름
    update.paragraphs[0].font.size = Pt(16)  # font size
    
    # 하단에 페이지 수
    page_layout = slide.shapes.add_textbox(left=Cm(22.06), top=Cm(26.34), width=Cm(2.8), height=Cm(0.94))
    page = page_layout.text_frame
    page.text = f'{len(prs.slides)}'
    page.paragraphs[0].font.size = Pt(16)
    page.paragraphs[0].alignment = PP_ALIGN.CENTER  # 가운데 정렬
    
    
    # 두 번째 페이지부터 공통으로 들어가는 조건
    #if i >= 1:
    # 왼쪽 하단 작성자
    writer_layout = slide.shapes.add_textbox(left=Cm(0.38), top=Cm(26.19), width=Cm(10.11), height=Cm(0.86))
    writer = writer_layout.text_frame
    writer.text = '작성자 : 개발임상 그룹 ' + name
    # title.text_frame.parhagraphs[0].font.name =    # font 이름
    writer.paragraphs[0].font.size = Pt(18)  # font size
        
    #return writer, update, page

In [10]:
def main():
    name = input('이름을 입력하세요 : ')
    date = input('날짜를 입력하세요 : ')

    # 전역 변수
    global prs

    # 슬라이드에 넣을 파일들을 임시로 지정해둠.
    #image_file = [[folder_path+'\\image\\b\\banana_01.png', folder_path+'\\image\\b\\banana_02.png']]
    #video_file = [[folder_path+'/image/b/banana01.png', folder_path+'/image/b/banana02.png']]

    # 슬라이드가 하나만 복사가 안되고 있음. len(sublist) = 2이기 때문이라서 이거 고치기
    # copy_slide02를 위치를 옮겨서 해결됨. 그런데 이미지가 같은 것이 2장 복사됨. 이거 고쳐야 함.
    #for sublist in image_file:
        #for img in sublist:
            #if sublist.find('.jpg'):

    for num in range(len(image_file)):
    #for sublist in video_path:
        copy_slide02(prs)  # 슬라이드 추가
        str_in(prs, name, date)     # 사용자 입력받은 값 넣기

        # 이미지 삽입

        if image_file[num][num].find('.jpg'):
            insert_image(prs, image_file)

 
        # 영상 삽입
        #if sublist.endswith('.mp4'):
            #create_video(prs, video_path)
            
    prs.save(folder_path + '/tmp.pptx')

In [11]:
if __name__ == '__main__':
    main()